In [ ]:
# An NFA accepting all strings that in 10 in substrig
class NFA:
    def __init__(self, states, alphabet, transitions, start_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions
        self.start_state = start_state
        self.accepting_states = accepting_states

    def accepts(self, string):
        current_states = {self.start_state}
        for char in string:
            next_states = set()
            for state in current_states:
                if (state, char) in self.transitions:
                    next_states.update(self.transitions[(state, char)])
            current_states = next_states
        return bool(current_states & self.accepting_states)

# Define the NFA components
states = {"q0", "q1", "q2"}
alphabet = {"0", "1"}
transitions = {
    ("q0", "0"): {"q0"},
    ("q0", "1"): {"q1"},
    ("q1", "0"): {"q2"},
    ("q1", "1"): {"q1"},
    ("q2", "0"): {"q2"},
    ("q2", "1"): {"q2"}
}
start_state = "q0"
accepting_states = {"q2"}

# Create the NFA instance
nfa = NFA(states, alphabet, transitions, start_state, accepting_states)

# Test some strings
print("01" , nfa.accepts("01"))      # True
print("101" , nfa.accepts("101"))     # True
print(  "1101" , nfa.accepts("1101"))    # True
print(  "001" , nfa.accepts("001"))     # True
print(  "10001" , nfa.accepts("10001"))   # True
print(  "111" , nfa.accepts("111"))     # False
print(  "0" , nfa.accepts("0"))       # False
print(  "1" , nfa.accepts("1"))       # False


### Alternative

In [ ]:
from automata.fa.nfa import NFA
from visual_automata.fa.nfa import VisualNFA

In [ ]:
# Define an visual_automata NFA that can accept any string with the pattern 10, 1010, 101010.

nfa = VisualNFA(
    states={"q0", "q1", "q2"},
    input_symbols={"0", "1"},
    transitions={
        "q0": {"": {"q2"}, "1": {"q1"}},
        "q1": {"1": {"q2"}, "0": {"q0", "q2"}},
        "q2": {},
    },
    initial_state="q0",
    final_states={"q0"},
)

In [ ]:
nfa.input_check("10100")

### Alternative

In [ ]:
from automata.fa.nfa import NFA
from automata.fa.dfa import DFA
import automata
# NFA which matches strings beginning with 'a', ending with 'a', and containing
# no consecutive 'b's
nfa = NFA(
    states={'q0', 'q1', 'q2'},
    input_symbols={'a', 'b'},
    transitions={
        'q0': {'a': {'q1'}},
        # Use '' as the key name for empty string (lambda/epsilon) transitions
        'q1': {'a': {'q1'}, '': {'q2'}},
        'q2': {'b': {'q0'}}
    },
    initial_state='q0',
    final_states={'q1'}
)

In [ ]:
import pandas as pd

def make_table(target_fa) -> pd.DataFrame:
    initial_state = target_fa.initial_state
    final_states = target_fa.final_states

    table = {}

    for from_state, to_state, symbol in target_fa.iter_transitions():
        # Prepare nice string for from_state
        if isinstance(from_state, frozenset):
            from_state_str = str(set(from_state))
        else:
            from_state_str = str(from_state)

        if from_state in final_states:
            from_state_str = "*" + from_state_str
        if from_state == initial_state:
            from_state_str = "→" + from_state_str

        # Prepare nice string for to_state
        if isinstance(to_state, frozenset):
            to_state_str = str(set(to_state))
        else:
            to_state_str = str(to_state)

        if to_state in final_states:
            to_state_str = "*" + to_state_str

        # Prepare nice symbol
        if symbol == "":
            symbol = "λ"

        from_state_dict = table.setdefault(from_state_str, dict())
        from_state_dict.setdefault(symbol, set()).add(to_state_str)

    # Reformat table for singleton sets
    for symbol_dict in table.values():
        for symbol in symbol_dict:
            if len(symbol_dict[symbol]) == 1:
                symbol_dict[symbol] = symbol_dict[symbol].pop()


    df = pd.DataFrame.from_dict(table).fillna("∅").T
    return df.reindex(sorted(df.columns), axis=1)


In [ ]:
input_str = 'abba'

In [ ]:
make_table(nfa)

In [ ]:
nfa.read_input_stepwise(input_str)

In [ ]:
if nfa.accepts_input(input_str):
    print('accepted')
    print(nfa.read_input(input_str))
else:
    print('rejected')